In [ ]:
source(paste0(dirname(dirname(getwd())),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

In [ ]:
library(forcats)
#library(wesanderson)

### 0 - Read exhaustive analysis

In [ ]:
fe <- fread(paste0(SHARE_DIR, "0_run_fe.csv"))
#lr <- fread(paste0(SHARE_DIR, "1_run_lr.csv"))

- Remove redundancies in Fisher exact output
- Some tests are equivalent but signs are flipped

In [ ]:
highlighter <- 
list(
 "Pan-Cancer ## Anti-PD-1" = "rna_geneset_Apm_lt75",
 "Breast ER+/HER- ## Pyrimidine" = "Cn_simple_chr13_q_gt25",
 "Pan-Cancer ## Pyrimidine" = "Cn_simple_chr18_p_gt25",
 "Pan-Cancer ## Anti-EGFR" = "rna_geneset_Hallmark_apical_surface_lt50",
 "Pan-Cancer ## Taxane" = "rna_geneset_Kegg_cytosolic_dna_sensing_pathway_lt75",
 "Breast ER+/HER- ## Aromatase inhibitor" = "Cn_simple_chr3_q_gt25",
 "Skin Melanoma ## Anti-PD-1" = "rna_geneset_Hallmark_interferon_alpha_response_lt50",
 "Pan-Cancer ## Multikinase inhibitor" = "Driver_total_gt50")

In [ ]:
remove_last <- function(i) i[-length(i)]

In [ ]:
fe_go <- 
fe %>% 
 rw() %>% 
 mu( raw = paste(remove_last(strsplit(feature, "_")[[1]]), collapse = "_")) %>% 
 ug() %>% 
 gb(cohortGo, raw, fisher_pval) %>% 
 mu( rk = row_number(desc(e_nr)), 
     label = factor(ifelse(pr_nr_given_e > pr_nr_overall, "Non-Response Associated", "Response Associated"), 
                   levels = c("Response Associated", "Non-Response Associated"))) %>%
 fi(rk == 1) %>% se(-rk) %>% 
 gb( cohortGo, label ) %>% 
 fi( !grepl("geneset_mp", feature), cohortGo != "Pan-Cancer ## Folinic acid ## Platinum ## Pyrimidine (ant)agonist ## Topoisomerase inhibitor") %>% 
 mu( or = (e_r+1)/(e_nr+1)/((ne_r+1)/(ne_nr + 1))) %>% 
 mu( cohortGo = gsub( " \\(ant\\)agonist", "", cohortGo)) %>% 
 rw() %>% 
 mu( size = ifelse( (tolower(cohortGo) %in% tolower(names(highlighter))) && 
                    (tolower(feature) %in% tolower(unlist(unname(highlighter)))) && 
                    fisher_pval < .001, 
                    5, 1), 
     size = ifelse( (cohortGo %in% "Pan-Cancer ## Pyrimidine") && 
                    (tolower(feature) %in%  "cn_simple_chr13_q_gt25"), 
                    1, size), 
    size2 = ifelse(size == 7, "big", "small")) %>% 
 ug()

- Adjust p-value

In [ ]:
fe_go$p_adj <- p.adjust(fe_go$fisher_pval, method = "BH")

In [ ]:
theshold = fe_go %>% gb(threshold = (p_adj < .05)) %>% mu(bh_max = max(fisher_pval)) %>% fi(threshold) %>% head(1) %>% pu(bh_max)

In [ ]:
theshold

In [ ]:
threshold

In [ ]:
options(repr.plot.width = 11.4, repr.plot.height = 6.4)

In [ ]:
go <- 
fe_go %>% 
 ggplot( aes(x = log(or), y = -log10(fisher_pval), fill = (or > 1), size = size), col = size2) + 
 geom_point(shape = 21) + 
 facet_wrap(~fct_rev(fct_reorder(str_wrap(cohortGo, width = 23), -log10(fisher_pval), .fun = max)), ncol = 7) +
 #facet_wrap(~ cohortGo, scales = "free_x", ncol = 7) +
 go_theme + 
 labs( x = "-Log (Odds Ratio) (Response vs Non-Response)", 
       y = "-Log10 (p-value)", 
       title = "Large scale testing of biomarkers for Non-Response") + 
 theme(legend.position = "none")+ 
 geom_hline(yintercept = -log10(.05), color = "grey", linetype = "dashed", size = .5) +
 geom_hline(yintercept = -log10(theshold), color = "grey", linetype = "dashed", size = .5) 

In [ ]:
ggsave( "/home/josephusset@vhio.org/biomarkers/util/figs/volcanoes.png", plot = go, width = 12, height = 7)

### Check the QQ plots